<a href="https://colab.research.google.com/github/machine-perception-robotics-group/JDLALectureNotebooks/blob/master/notebooks/xx_ordinal_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによる順序回帰

---

## 目的

ニューラルネットワークを使用して，順序回帰問題を解く．

## モジュールのインポート

プログラムの実行に必要なモジュールをインポートします．


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import csv

## データセットのダウンロード

今回使用するデータセットはWine Quarity Datasetです．

このデータセットのファイルをダウンロードし，zipファイルを解凍します．

[Wine Quality Dataset](https://archive.ics.uci.edu/dataset/186/wine+quality)

In [ ]:
import gdown
gdown.download('https://drive.google.com/uc?id=1vq_HOkwUl9D3SM4aXRcnevBYw3_ZcjJ7', 'WineQT.zip', quiet=False)
! unzip -q WineQT.zip

CSVファイルの中身を確認してみます．



In [ ]:
with open("WineQT.csv", "r") as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(row)
        if i > 3:
            break

これを踏まえて，データセットクラスを作成します．

In [ ]:
class WineDataset(Dataset):
  def __init__(self, filepath):
    self.data = []
    self.label = []
    with open(filepath, "r") as f:
      reader = csv.reader(f)
      next(reader)  # header をスキップ
      for row in reader:
        self.data.append([float(cell) for cell in row[:-2]])  # 最後の列以外を特徴量として扱う
        self.label.append(int(row[-2]))  # 最後から2番目の列をラベルとして扱う．

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    data = torch.tensor(self.data[idx], dtype=torch.float32)
    label = torch.tensor(self.label[idx], dtype=torch.long)
    return data, label

## ネットワーククラスの作成

ここでは全結合層からなるニューラルネットワークを作成します．

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()

        self.l1 = nn.Linear(input_dim, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.relu(x)
        x = self.l3(x)
        return x

## 学習

ネットワークを学習します．

In [ ]:
# データセット，データローダーの作成
train_dataset = WineDataset("WineQT.csv")
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# モデル，損失関数，最適化関数の作成
model = MLP(input_dim=11, output_dim=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 学習
num_epochs = 10
for epoch in range(num_epochs):
  running_loss = 0.0
  for i, (data, labels) in enumerate(train_dataloader):
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataloader)}")